<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/extractive_summarization_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install rouge-score
!pip install fasttext
!pip install compress-fasttext
!pip install gensim==3.8.3
!pip install import-ipynb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#!unzip "/content/drive/MyDrive/data/glove*.zip"

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import string
import re
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import networkx as nx
from rouge_score import rouge_scorer
import gensim
import fasttext
from gensim.models import FastText
import compress_fasttext
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords  
import import_ipynb

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load in functions from extract_summarization notebook
%cd "drive/MyDrive/Colab Notebooks"
from extractive_summarization import *

/content/drive/MyDrive/Colab Notebooks
importing Jupyter notebook from extractive_summarization.ipynb
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def main():

  df = data_setup()
  tfidf, feature_array = corpus_tfidf(df) 
  embeddings = load_embeddings()

  for model in ['lsa']: #['baseline', 'textrank', 'lsa']:
    #  train each configuration on a subset of the data and get evaluation metrics (1/10 of 10,000)
    eval_results, _ = train_config_loop(df.head(100), tfidf, feature_array, embeddings, model_configurations[model][0:60],
                                        save_every_cnt = 50, start_over = True, filename = 'train_config_loop') # todo 1000 and _randomize_

    # find best config for each evaluation metrics
    best_configs = find_best_configs(eval_results)

    # train full model on the best configurations for each metric
    print('training best models')
    eval_results_dict = {}
    model_results_dict = {}
    seen_configs = {} 
    for metric in best_configs.keys(): 
      config = tuple(best_configs[metric].strip('(').strip(')').replace("'", "").split(', '))
      if config not in seen_configs.keys():
        eval_results, model_results = train_config_loop(df, tfidf, feature_array, embeddings, [config], save_every_cnt = 1, start_over = True, filename = 'best_config_loop')
        eval_results_dict[metric] = eval_results[str(config)][metric]
        model_results_dict[metric] = model_results[str(config)]
        seen_configs[config] = metric
      # prevent duplicative retraining: use existing results if best config for prior metric
      else:
        eval_results_dict[metric] = eval_results_dict[seen_configs[config]]
        model_results_dict[metric] = model_results_dict[seen_configs[config]]

    # save best models
    with open('/content/drive/MyDrive/data/trained_model_' + model + '.pkl', 'wb') as f:
        pickle.dump(eval_results_dict, f) # for each eval metric, distribution of evaluation metrics 
        pickle.dump(model_results_dict, f) # for each eval metric, data with predicted summaries
        pickle.dump(best_configs, f) # for each eval metric, best configuration 

In [7]:
# TODO set up stop points in long runs - timeout or memory issues (pr)
# save results in chunks and append to existing data structures 

In [8]:
main()

TypeError: ignored